In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# pd.set_option('display.max_colwidth', None)

In [3]:
pd.reset_option('display.max_colwidth')

In [4]:
collected_08_23 = {
    "otodom_krk" : "../original_data/collected_08_23/master_data/otodom_krk.json",
    "otodom_gda" : "../original_data/collected_08_23/master_data/otodom_gda.json",
    "olx_gda" : "../original_data/collected_08_23/master_data/olx_gda.json",
    "olx_krk" : "../original_data/collected_08_23/master_data/olx_krk.json",
    "gratka_gda" : "../original_data/collected_08_23/master_data/gratka_gda.json",
    "gratak_krk" : "../original_data/collected_08_23/master_data/gratka_krk.json",
    "trojmiasto_gda" : "../original_data/collected_08_23/master_data/trojmiasto_gda.json"
}

In [5]:
# Create a dictionary to store DataFrames
dataframes_08_23 = {}

# Iterate through the JSON data and create DataFrames
for key, value in collected_08_23.items():
    dataframes_08_23[key] = pd.read_json(value)  # Assuming the data is in JSON format

# Now, you have a dictionary of DataFrames where each key is a dataset
# You can access them like this:
otodom_krk_df = dataframes_08_23["otodom_krk"]
otodom_gda_df = dataframes_08_23["otodom_gda"]
olx_gda_df = dataframes_08_23["olx_gda"]
olx_krk_df = dataframes_08_23["olx_krk"]
gratka_gda_df = dataframes_08_23["gratka_gda"]
gratak_krk_df = dataframes_08_23["gratak_krk"]
trojmiasto_gda_df = dataframes_08_23["trojmiasto_gda"]

In [9]:
gratka_gda_df.shape

(354, 31)

In [11]:
trojmiasto_gda_df.isna().sum()

link                          0
rent                          0
description                   0
address                       0
additional_info               0
Liczba pokoi                  0
Piętro                        4
Rok budowy                  122
Jestem                        0
Rodzaj nieruchomości          0
Umeblowane                    3
Liczba pięter w budynku      28
Powierzchnia                  0
Kaucja                      648
Typ ogrzewania              456
Niezależne pokoje           896
Dostępne od                 649
Liczba osób w pokoju       1009
Nr oferty                   765
Nr licencji pośrednika     1064
Powierzchnia działki       1159
dtype: int64

In [436]:
heating_mapping = {
    'Centralne': 'Centralne',
    'Miejskie': 'Miejskie',
    'Co Miejskie': 'Centralno-miejskie',
    'Elektryczne': 'Elektryczne',
    'C. o. miejskie': 'Centralno-miejskie',
    'Co Własne': 'Centralne',
    '(Nieznane)': 'Inne',
    'Gazowe': 'Gazowe',
    'Ogrzewanie miejskie': 'Miejskie',
    'Kotłownia w bloku': 'Inne',
    'CO miejskie': 'Centralno-miejskie',
    'Kominek': 'Inne',
    'C/o': 'Centralne',
    'Cenyralne': 'Centralne',
    'C. o': 'Centralne',
    'Podłogowe wliczone w cenę': 'Inne',
    'Centralne, podłogowe': 'Centralne',
    'Gazowe, klimatyzator': 'Gazowe',
    'Centralne miejskie': 'Centralno-miejskie',
    'Centralny': 'Centralne',
    'Centralne, miejskie': 'Centralno-miejskie',
    'C. o.': 'Centralne',
    'C. O. miejskie': 'Centralno-miejskie',
    'Kaloryfery': 'Inne',
    'Centralne ogrzewanie': 'Centralne',
    'CO własne': 'Inne',
    'Centralne- z sieci': 'Centralno-miejskie',
    'GPEC': 'Inne',
    'Gaz': 'Gazowe',
    'Gazowy': 'Gazowe',
    'Miejskie (centralne)': 'Centralno-miejskie',
    'C. O.': 'Centralne',
    'Elektryczny': 'Elektryczne',
    'Gaz wspólnota': 'Gazowe',
    'Kotłowania': 'Inne',
    'Elektryczne akumulacyjne (II taryfa)': 'Elektryczne',
    'Ogrzrwanie centralne': 'Centralne',
    'Miejskie (gpc)': 'Miejskie',
    'OC': 'Centralne',
    'Miejski podłogowe': 'Miejskie',
    'Co miejskie': 'Centralno-miejskie',
    'Wentylacja wymuszona': 'Inne',
    'Eco Groszek': 'Inne'
}

furnished_mapping = {
    'Tak': True,
    'Nie': False
}

private_seller = {
    'Firmą': False,
    'Osobą prywatną': True
}

dict_regions = {
  "Brzeźno": "Brzeźno",
  "Morena": "Morena",
  "Dolne": "Śródmieście",
  "Śródmieście": "Śródmieście",
  "Orunia": "Orunia",
  "Przymorze": "Przymorze",
  "Wrzeszcz": "Wrzeszcz",
  "Letnica": "Letnica",
  "Żabianka": "Żabianka",
  "Jasień": "Jasień",
  "Stogi": "Stogi",
  "Zakoniczyn": "Południe",
  "Wrzwszcz": "Wrzeszcz",
  "Stare": "Śródmieście",
  "Długie": "Śródmieście",
  "Łostowice": "Łostowice",
  "Aniołki": "Aniołki",
  "Suchanino": "Morena",
  "Kowale": "Południe",
  "Nowy": "Brzeźno",
  "Oliwa": "Oliwa",
  "Ujeścisko": "Łostowice",
  "Siedlce": "Siedlce",
  "Rebiechowo": "Matarnia",
  "Piecki": "Morena",
  "Jelitkowo": "Jelitkowo",
  "Jelitkowski": "Jelitkowo",
  "Południe": "Południe",
  "Zaspa": "Zaspa",
  "Ul.": "Wrzeszcz",
  "Siedle": "Siedlce",
  "Matarnia-": "Matarnia",
  "Osowa": "Osowa",
  "Antoniego": "Chełm",
  "Kokoszki": "Matarnia",
  "Przeróbka": "Stogi",
  "Strzyża": "Wrzeszcz",
  "Chełm": "Chełm",
  "Osiedle": "Żabianka",
  "Młode": "Stogi",
  "Podwale": "Śródmieście",
  "Czarny": "Brzeźno",
  "Długie": "Śródmieście",
  "Morena/": "Morena",
  "Obrońców": "Przymorze",
  "Piecki-": "Morena",
  "Maćkowy": "Południe",
  "G": "Wrzeszcz",
  "Morenagen.": "Morena",
  "Łostwice": "Łostowice",
  "Kiełpinek": "Matarnia",
  "V": "Śródmieście",
  "Nowa": "Matarnia",
  "Angielska": "Śródmieście",
  "Letnica37": "Letnica",
  "Sródmieście": "Śródmieście",
  "Karczemki": "Matarnia",
  "Cygańska": "Aniołki",
  "Ofiar": "Aniołki",
  "Wrzeszczal.": "Wrzeszcz",
  "Wzgórze": "Chełm",
  "Bora-": "Wrzeszcz",
  "Starówka": "Śródmieście",
  "Przymorzeul.": "Przymorze",
  "Suchaninovan": "Morena",
  "Ujeścisko-": "Łostowice",
  "Garnizon": "Wrzeszcz",
  "Przymorze10": "Przymorze"
}

In [437]:
def fill_na_with_mean(column_name: str, data_frame: pd.DataFrame):
    mean = data_frame[column_name][data_frame[column_name].notna()].astype(int).mean()
    data_frame.loc[:, column_name] = data_frame[column_name].fillna(mean)
    data_frame.loc[:, column_name] = data_frame[column_name].astype(int)
    return data_frame

def extract_first_word(given_string: str):
    big_letter = 0
    for idx, letter in enumerate(given_string):
        if letter.isupper() and big_letter == 1:
            return given_string[:idx]
        elif letter.isupper():
            big_letter += 1
    return given_string

def clear_trojmiasto(df: pd.DataFrame):
    # columns_to_drop
    df.drop(
        labels=[
                'description', 
                'Dostępne od', 
                'Nr licencji pośrednika', 
                'Powierzchnia działki', 
                'Nr oferty',
                'Niezależne pokoje',
                'Liczba osób w pokoju',
                'Kaucja',
                'Rodzaj nieruchomości'
        ],
                axis=1, inplace=True)
    
    # rent variable
    df['rent'] = df['rent'].apply(lambda x: float(x.replace("zł", "").replace(",",".").replace(" ","")))

    # floor
    df['Piętro'] = df['Piętro'].replace('Parter', '0')
    df = fill_na_with_mean(column_name="Piętro", data_frame=df)
    df = df.rename(mapper={"Piętro": "floor"}, axis=1)
    
    # rooms
    df = df[df['Liczba pokoi'] <= 5]
    df.rename(mapper={"Liczba pokoi": "rooms"}, axis=1, inplace=True)

    # floors_in_building
    df = fill_na_with_mean(column_name= "Liczba pięter w budynku", data_frame=df)
    df.rename(mapper={'Liczba pięter w budynku': "floors_in_building"}, axis=1, inplace=True)

    # build_year
    df = df[df['Rok budowy'] >= 1850]
    df = fill_na_with_mean(column_name="Rok budowy", data_frame=df)
    df.rename(mapper={'Rok budowy': "build_year"}, axis=1, inplace=True)

    # heating_type
    df['Typ ogrzewania'] = df['Typ ogrzewania'].map(heating_mapping)
    df['Typ ogrzewania'].fillna('Inne', axis=0, inplace=True)
    df.rename(mapper={'Typ ogrzewania': "heating_type"}, axis=1, inplace=True)

    # furnished
    df['Umeblowane'] = df['Umeblowane'].map(furnished_mapping)
    df['Umeblowane'].fillna(True, axis=0, inplace=True)
    df.rename(mapper={'Umeblowane': 'furnished'}, axis=1, inplace=True)

    # surface
    df['Powierzchnia'].str.replace(',','.').astype(float)
    df.rename(mapper={'Powierzchnia': 'surface'}, axis=1, inplace=True)

    # private seller
    df['Jestem'] = df['Jestem'].map(private_seller)
    df.rename(mapper={'Jestem': 'private_seller'}, axis=1, inplace=True)

    # address
    df['address2'] = df['address'].str.strip().str.replace(" ","").str.replace("Gdańsk", "").str.replace("gdańsk", "").str.lstrip("\xa0").apply(extract_first_word)
    df['address2'] = df['address2'].map(dict_regions)

    return df

In [438]:
trojmiasto_gda_df = clear_trojmiasto(df = trojmiasto_gda_df)

In [462]:
trojmiasto_gda_df[trojmiasto_gda_df['address2'].isna()]['address'].value_counts()

address
Gdańsk                              8
GdańskOgarna                        6
Gdańsk BrzeźnoGdańska               4
GdańskHallera                       3
GdańskSzafarnia 11                  2
GdańskStępkarska                    2
gdańsk wrzeszcz                     2
GdańskKowalska                      2
GdańskGłęboka                       1
Gdańsk Solikowskiego 3a             1
Gdańsk Piecki-migowo                1
Gdańsk Stogiskiby 24                1
Gdańsk Śródmieściewałowa            1
GdańskWałowa                        1
Gdańsk Letnica 2                    1
Gdańsk Jasieńlawendowe wzgorze      1
Gdańsk Osiek                        1
GdańskGdyńska                       1
GdańskZamiejska 50                  1
Gdańsk Wałowa25d                    1
Gdańsk Kochanowskiego41             1
Gdańsk Porębskiego                  1
Gdańsk Starowiejska 91              1
Gdansk                              1
Gdańsk Gospody                      1
GdańskKotwiczników                  1
Gdań

In [409]:
trojmiasto_gda_df['address2'].str.split("k")[1:]

1           [Brzeźno]
2           [Brzeźno]
3            [Morena]
4       [Śródmieście]
5       [Śródmieście]
            ...      
1177      [Przymorze]
1178    [Śródmieście]
1179        [Siedlce]
1180    [Śródmieście]
1181          [Osowa]
Name: address2, Length: 1056, dtype: object

In [388]:
trojmiasto_gda_df['address2'].isna().sum()

71

In [228]:
district = re.findall('[A-Z][^A-Z]*', str(trojmiasto_gda_df['address'].str.replace("Gdańsk", "").str.replace(" ","")))

In [334]:
trojmiasto_gda_df['address_tied'] = trojmiasto_gda_df['address'].str.replace("Gdańsk", "").str.replace(" ","").str.lstrip("\xa0").apply(extract_first_word)

In [335]:
trojmiasto_gda_df['address_tied'].map(dict_regions).value_counts()

address_tied
Śródmieście    233
Przymorze      177
Wrzeszcz       132
Łostowice       51
Morena          46
Brzeźno         43
Jasień          37
Letnica         31
Żabianka        30
Oliwa           30
Zaspa           26
Południe        23
Stogi           20
Chełm           20
Orunia          18
Jelitkowo       17
Matarnia        16
Siedlce         15
Aniołki         11
Osowa           10
Name: count, dtype: int64

In [276]:
dict_count = {}

for k, v in dict_regions.items():
    if v not in dict_count.keys():
        dict_count[v] = 1
    if v in dict_count.keys():
        dict_count[v] += 1

In [332]:
dict_regions = {
  "Brzeźno": "Brzeźno",
  "Morena": "Morena",
  "Dolne": "Śródmieście",
  "Śródmieście": "Śródmieście",
  "Orunia": "Orunia",
  "Przymorze": "Przymorze",
  "Wrzeszcz": "Wrzeszcz",
  "Letnica": "Letnica",
  "Żabianka": "Żabianka",
  "Jasień": "Jasień",
  "Stogi": "Stogi",
  "Zakoniczyn": "Południe",
  "Wrzwszcz": "Wrzeszcz",
  "Stare": "Śródmieście",
  "Długie": "Śródmieście",
  "Łostowice": "Łostowice",
  "Aniołki": "Aniołki",
  "Suchanino": "Morena",
  "Kowale": "Południe",
  "Nowy": "Brzeźno",
  "Oliwa": "Oliwa",
  "Ujeścisko": "Łostowice",
  "Siedlce": "Siedlce",
  "Rebiechowo": "Matarnia",
  "Piecki": "Morena",
  "Jelitkowo": "Jelitkowo",
  "Jelitkowski": "Jelitkowo",
  "Południe": "Południe",
  "Zaspa": "Zaspa",
  "Ul.": "Wrzeszcz",
  "Siedle": "Siedlce",
  "Matarnia-": "Matarnia",
  "Osowa": "Osowa",
  "Antoniego": "Chełm",
  "Kokoszki": "Matarnia",
  "Przeróbka": "Stogi",
  "Strzyża": "Wrzeszcz",
  "Chełm": "Chełm",
  "Osiedle": "Żabianka",
  "Młode": "Stogi",
  "Podwale": "Śródmieście",
  "Czarny": "Brzeźno",
  "Długie": "Śródmieście",
  "Morena/": "Morena",
  "Obrońców": "Przymorze",
  "Piecki-": "Morena",
  "Maćkowy": "Południe",
  "G": "Wrzeszcz",
  "Morenagen.": "Morena",
  "Łostwice": "Łostowice",
  "Kiełpinek": "Matarnia",
  "V": "Śródmieście",
  "Nowa": "Matarnia",
  "Angielska": "Śródmieście",
  "Letnica37": "Letnica",
  "Sródmieście": "Śródmieście",
  "Karczemki": "Matarnia",
  "Cygańska": "Aniołki",
  "Ofiar": "Aniołki",
  "Wrzeszczal.": "Wrzeszcz",
  "Wzgórze": "Chełm",
  "Bora-": "Wrzeszcz",
  "Starówka": "Śródmieście",
  "Przymorzeul.": "Przymorze",
  "Suchaninovan": "Morena",
  "Ujeścisko-": "Łostowice",
  "Garnizon": "Wrzeszcz",
  "Przymorze10": "Przymorze"
}

In [327]:
trojmiasto_gda_df.isna().sum()

link                    0
rent                    0
address                 0
additional_info         0
rooms                   0
floor                   0
build_year              0
private_seller          0
Rodzaj nieruchomości    0
furnished               0
floors_in_building      0
surface                 0
heating_type            0
address_tied            0
dtype: int64

In [34]:
trojmiasto_gda_df.columns

Index(['link', 'rent', 'description', 'address', 'additional_info',
       'Liczba pokoi', 'Piętro', 'Rok budowy', 'Jestem',
       'Rodzaj nieruchomości', 'Umeblowane', 'Liczba pięter w budynku',
       'Powierzchnia', 'Kaucja', 'Typ ogrzewania', 'Niezależne pokoje',
       'Dostępne od', 'Liczba osób w pokoju', 'Nr oferty',
       'Nr licencji pośrednika', 'Powierzchnia działki'],
      dtype='object')

In [38]:
trojmiasto_gda_df['Jestem'].unique()


array(['Firmą', 'Osobą prywatną'], dtype=object)

In [30]:
trojmiasto_gda_df['rent']

0        3 200 zł
1        3 700 zł
2        3 000 zł
3        3 200 zł
4        2 200 zł
          ...    
1178     5 800 zł
1179     3 500 zł
1180    12 000 zł
1181     3 500 zł
1182     2 700 zł
Name: rent, Length: 1183, dtype: object

In [463]:
trojmiasto_gda_df.columns

Index(['link', 'rent', 'address', 'additional_info', 'rooms', 'floor',
       'build_year', 'private_seller', 'furnished', 'floors_in_building',
       'surface', 'heating_type', 'address2'],
      dtype='object')